In [2]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D


def radar_factory(num_vars, frame='circle'):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def plot_radar_chart(ax, theta, metrics, values, algorithms, colors):
    metrics = list(metrics)
    values = values[1:]
    values[4,:] = values[4,:]/2
    
    num_metrics = len(metrics)

    # Plot the filled radar map
    ax.set_rgrids([0.2, 0.4, 0.6, 0.8])
    
    for algorithm, color in zip(algorithms, colors):
        algorithm_index = algorithms.index(algorithm)
        ax.plot(theta, values[:,algorithm_index],label=algorithm, color=color)
        ax.fill(theta, values[:,algorithm_index], facecolor=color, alpha=0.25, label='_nolegend_')
    # Set the angle labels
    ax.set_varlabels(metrics)
    
# List of algorithm files
path = 'D:/Pranav/Results/CURE-OR'
algorithm_files = [path + '/AD.csv', path + '/AGC.csv', path + '/MAXIM.csv']
algorithms = ['AD', 'AGC', 'Maxim']
# Set up colors for each algorithm
algorithm_colors = {'AD': 'red', 'AGC': 'green', 'Maxim': 'blue'}
df = pd.read_csv(algorithm_files[0])
noise_types = df.iloc[::4, 0].tolist()
# Iterate through each noise type
for noise_type in noise_types:
    noise_name, _ = noise_type.split('-', 1)
    
    N = 6
    theta = radar_factory(N, frame='polygon')

    fig, ax = plt.subplots(figsize=(9, 9), nrows=1, ncols=1,
                            subplot_kw=dict(projection='radar'))
    ax.set_title(f'Radar Chart - {noise_name}', va='bottom')
    ax.set_title(noise_name.capitalize(), weight='bold', size='medium', position=(0.5, 1.1),
                    horizontalalignment='center', verticalalignment='center')

    

    # Iterate through each algorithm
    colors = []
    algorithm_values = []
    for i, algorithm_file in enumerate(algorithm_files):
        # Read data from CSV for the current algorithm
        algorithm =  algorithms[i]
        df = pd.read_csv(algorithm_file)
        metrics_to_normalize = df.columns[3] 

        # Filter rows for the current noise type and algorithm
        subset = df[df.iloc[:, 0].str.startswith(noise_name)]

        # Extract metrics
        algorithm_metrics = subset.iloc[:, 1:].apply(lambda x: eval(x.str.replace('/', ',').values[0])).iloc[1,:]
        # Normalize PSNR values to be on the same scale as other metrics (0 to 1)

        colors.append(algorithm_colors[algorithm])
        algorithm_values.append(algorithm_metrics.values.flatten())

    # Plot radar chart
    plot_radar_chart(ax, theta, df.columns[2:], np.array(algorithm_values).T, algorithms, colors)

    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    # Save the figure
    output_directory = path
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    output_filename = os.path.join(output_directory, f'Polygonal_Radar_Map_{noise_name}.png')
    plt.savefig(output_filename, bbox_inches='tight')
    plt.close()

    # Show radar chart for the current noise type
    plt.show()
